In [1]:
"""
Created on Mar 06 2022
@author: Jerónimo Arenas García

Temporary routine for generation of JSON files
simulating the creation of files required for the
BI tool

If other datasets want to be ingested, we need to
1) Change the SELECT statement accordingly
2) Change the fields as necessary
3) Change the name of the folder where the JSON will be save
"""

import TMinferencer
import io
from langdetect import detect
from pyspark.sql import SparkSession
from pyspark.sql.functions import array_contains, concat_ws, col, udf
from pyspark.sql.types import StringType

In [2]:
sc.addPyFile("TMinferencer.py")

In [3]:
# Read subtable from parquet file
S2papers = spark.sql("SELECT id, year, venue, title, paperAbstract FROM parquet.`/export/ml4ds/IntelComp/Datalake/SemanticScholar/20220201/papers.parquet` where array_contains(fieldsOfStudy, 'Computer Science')")

22/03/06 22:03:50 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/03/06 22:03:50 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist
22/03/06 22:03:52 WARN metastore.ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
22/03/06 22:03:52 WARN metastore.ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore jarenas@192.168.148.225
22/03/06 22:03:52 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
22/03/06 22:03:52 WARN metastore.ObjectStore: Failed to get database parquet, returning NoSuchObjectException


In [4]:
#Concatenate text fields to lemmatize
S2papers = (
    S2papers.withColumn("rawtext",concat_ws('. ', "title", "paperAbstract"))
    .drop("paperAbstract")
)

def my_detect(rawt):
    try:
        return detect(rawt)
    except:
        return "na"

udf_detect = udf(lambda x:my_detect(x), StringType() )
S2papers = S2papers.withColumn("language",udf_detect(col("rawtext")))
S2papers = (
    S2papers.filter(col("language")=="en")
    .drop("language")
)


In [5]:
def TMinference(rawt):
    thetas = TMinferencer.main(io.StringIO(rawt.replace("\n", " ")))
    return " ".join("t"+str(el[0])+"|"+str(round(1000*el[1])) for el in thetas[0].items())
udf_TMinference = udf(lambda x:TMinference(x), StringType() )

S2papers = S2papers.withColumn("TM40",udf_TMinference(col("rawtext")))
S2papers = S2papers.drop("rawtext")


In [6]:
#Save corpus
S2papers.repartition(100).write.json(f"file:///export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/S2CS.json")